# <center>  **<span style="font-size:60px;">Deterministic Annealing en Clustering</span>** </center>

# <center> Universidad de Alicante </center>
<p align="center"><img src=".ipynb_gfx/UA.png" width="330" height="190"></p>
<br>

**Nombre:** Santiago Álvarez Geanta <br>
**Fecha:** 29 de marzo de 2025 <br>
**Grado:** Ingeniería en Inteligencia Artificial <br>
**Grupo:** 1 <br>


### Profesores
- Francisco Escolano Ruíz
- Ahmed Begga Hachlafi
<br>

---
# Introducción
---

El algoritmo de Deterministic Annealing (DA) es una técnica de optimización inspirada en el annealing, pero aplicada al problema de clustering. Su interés radica en la capacidad de encontrar agrupaciones óptimas sin depender de la inicialización aleatoria de los clusters. La idea central de DA es controlar la **temperatura** del sistema para explorar el espacio de soluciones de manera más estable y evitar caer en óptimos locales.

---
# Fundamentos del Algoritmo
---

El proceso de **Deterministic Annealing** sigue una estrategia iterativa que combina exploración y refinamiento a través de la manipulación de la "temperatura". Este algoritmo, inspirado en el proceso físico de recocido, ajusta la rigidez de las asignaciones a lo largo de las iteraciones, lo que le permite evitar quedar atrapado en soluciones subóptimas. A lo largo de las iteraciones, la temperatura disminuye, lo que facilita la convergencia hacia una solución óptima. Es como un proceso de "enfriamiento controlado" que mejora la precisión de las asignaciones, pero al mismo tiempo permite una fase inicial de exploración.

## Expectation: Cálculo de Probabilidades de Pertenencia

$$ M_{ai} = \frac{e^{-D(x_a, c_i)/T}}{\sum_{i'=1}^{k} e^{-D(x_a, c_{i'})/T}} $$

#### Explicación de la Fórmula:
- **Distancia** $D(x_a, c_i)$: Mide cuán lejos está un punto del cluster. La elección de la distancia tiene un impacto directo sobre cómo se forman los clusters. En nuesto caso, al calcular la matriz de distancias euclidianas al cuadrado entre puntos y clusters, los puntos tienden a agruparse en clusters de **forma circular**.
$$ ||a-b||² = ||a||² + ||b||² - 2<a,b> $$

  
- **Exponencial negativa** $e^{-D(x_a, c_i)/T}$: Este término convierte la distancia en una probabilidad relativa, y su comportamiento tiene implicaciones importantes:
  - Si el punto está **cerca** de un cluster, su probabilidad de pertenencia será **alta**. La relación exponencial asegura que las distancias más cortas dominan fuertemente la asignación.
  - Si el punto está **lejísimos** de un cluster, la probabilidad será muy **baja**. La rapidez con la que decae esta probabilidad depende de la temperatura.
  
  - **Casos Extremos**: Si la temperatura $T$ es extremadamente alta, incluso los puntos muy lejanos a un cluster pueden tener una probabilidad significativa de pertenecer a él. Esto ayuda a evitar que el algoritmo se quede "atrapado" demasiado pronto en una asignación subóptima, promoviendo una fase de exploración más amplia al principio del proceso. Sin embargo, si la temperatura es baja, el algoritmo se vuelve **muy determinista**, y un solo cluster puede absorber todos los puntos cercanos sin permitir mucha variabilidad.

- **División por la suma de exponentes**: Este paso asegura que todas las probabilidades sumen **1**, lo que convierte el valor en una distribución válida de probabilidades.

- **Temperatura** $T$: La temperatura es el parámetro clave que controla la "rigidez" de las asignaciones. A medida que la temperatura disminuye, el algoritmo se vuelve más selectivo. Es decir, empieza a asignar puntos con más certeza y menos flexibilidad.
  
  - **Alta temperatura** ($T$ grande): Al principio, cuando la temperatura es alta, el algoritmo favorece una **exploración amplia**, permitiendo que los puntos puedan pertenecer a más de un cluster. Esto evita que el modelo se quede atrapado en soluciones no globales y promueve una mayor diversidad en la asignación.

  - **Baja temperatura** ($T$ pequeña): Conforme la temperatura disminuye, las asignaciones se vuelven más **deterministas**. En este estado, el algoritmo comienza a realizar ajustes más finos, favoreciendo una separación clara entre los clusters. Esto permite que el algoritmo ajuste su salida de manera precisa, sin la interferencia de "soluciones" ambiguas.

**Ejemplo práctico**: Imagina que estamos agrupando clientes según sus preferencias de compra. Al principio, los puntos (clientes) pueden estar distribuidos aleatoriamente, y el algoritmo permitirá que algunos clientes puedan estar asignados a más de un grupo (probabilidad distribuida entre varios clusters). A medida que la temperatura baja, los clientes se asignan de manera más definitiva a un solo grupo, representando de forma precisa su preferencia dominante.

### Update: Reajuste de clusters

Los clusters se recalculan como la media ponderada de los puntos asignados, donde los pesos corresponden a las probabilidades de pertenencia:

$$ c_j = \frac{\sum_i p(i|j) x_i}{\sum_i p(i|j)} $$

Este paso asegura que los clusters se muevan de manera coherente hacia las regiones densas de los puntos asignados. A medida que la temperatura disminuye, el sistema se vuelve más selectivo, refinando las asignaciones de los puntos y ajustando las posiciones de los clusters.

**Curiosidad sobre el comportamiento del ajuste de clusters**: Al principio, los clusters pueden ser influenciados por puntos muy lejanos debido a la alta temperatura. Sin embargo, conforme la temperatura baja, estos efectos se atenúan, y los clusters se mueven hacia las "verdaderas" regiones densas de los datos.

---
# Casos de Fallo
---

**1. Descenso Rápido de Temperatura:**
- Si la temperatura disminuye demasiado rápido, el algoritmo puede perder la capacidad de explorar adecuadamente el espacio de soluciones. Esto ocurre porque la probabilidad de aceptar soluciones peores disminuye rápidamente, lo que lleva al algoritmo a quedar atrapado en un mínimo local. Provocando que se forman clusters incorrectos, ya que el algoritmo no explora suficientemente el espacio de búsqueda para encontrar una solución globalmente óptima. Así que en su defecto, acaba aceptando soluciones **subóptimas**.

**2. Descenso Lento de Temperatura:**
- Si la temperatura baja demasiado lento, el algoritmo se vuelve computacionalmente costoso. Aunque el algoritmo tiene más tiempo para explorar (aunque en nuestro caso lo limitamos a 100 iteraciones), al aceptar muchas soluciones subóptimas, el algoritmo reduce la varianza entre los clusters, haciendo que los datos se agrupen en un solo cluster grande en lugar de varios más pequeños y definidos.

<p align="center">
  <img src="./report_gfx/NormalTemperature.png" width="30%" />
  <img src="./report_gfx/FastTemperature.png" width="30%" />
  <img src="./report_gfx/SlowTemperature.png" width="30%" />
</p>


**3. Inicialización de clusters**:

Cuando inicializamos todos los clusters en el baricentro de los datos, todos los clusters empiezan en la misma posición. Esto implica que al principio los clusters están en la misma ubicación, lo que puede llevar a que en las primeras iteraciones no se formen múltiples clusters.
* self.centroids = np.mean(X, axis=0, keepdims=True)
* self.centroids = np.repeat(self.centroids, self.n_clusters, axis=0)

Una solución es inicializar los clusters de forma aleatoria, lo que ayuda a evitar el problema de que todos los clusters se agrupen en el mismo punto.
* self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]

<p align="center"> 
    <img src="./report_gfx/AleatoryInit.png" width="30%"/>
    <img src="./report_gfx/BaricenterInit.png" width="30%"/>
</p>

**4. Número de iteraciones**

* **Número de iteraciones bajo:**<br>
Si el número de iteraciones es demasiado bajo, el algoritmo no tendrá suficiente tiempo para explorar el espacio de solución adecuadamente. Esto puede llevar a una convergencia prematura antes de que el sistema haya llegado a una solución estable o óptima. Es decir, los clusters no se ajustarán bien a los datos, y quedarán centrados en un punto central dando así soluciones imprecisas.

* **Número de iteraciones:**<br>
Si el número de iteraciones es muy alto, el algoritmo seguirá funcionando durante mucho más tiempo de lo necesario, lo cual no necesariamente mejora la calidad del resultado, ya que la temperatura puede disminuir a un ritmo tan lento que no implica mejora alguna. Ocasionando así una pérdida de tiempo sin apenas ganancia de resultados. Esimportante destacar que la solución que obtendremos será más precisa que al utilizar un menor número de iteraciones.

<p align="center"> 
    <img src="./report_gfx/LowIterations.png" width="30%"/>
    <img src="./report_gfx/HighIterations.png" width="30%"/>
</p>

**5. Varianza de los datos generados**

* **Valor de varianza bajo:**<br>
Cuando los datos tienen una varianza baja, los puntos de datos se agrupan estrechamente alrededor de la media. Esto implica que los valores no se dispersan significativamente, lo que resulta en una estructura de clusters más compacta y con poca separación entre los puntos. Como vemos en la imagen, el algoritmo encuentre pocos clusters bien definidos, ya que los puntos tienen una probabilidad alta de pertenecer a los mismos clusters. Por tanto, al algoritmo le cuesta distinguir entre clusters distintos si la dispersión es tan pequeña que la separación no es clara.

<p align="center">  <img src="./report_gfx/LowStd.png" width="50%"/> </p>

* **Valor de varianza normal:**<br>
Con una varianza normal, los datos están distribuidos de manera más equilibrada, lo que permite que el algoritmo de clustering encuentre estructuras de clusters definidas pero no excesivamente dispersas. En este caso, los puntos de datos están lo suficientemente separados para que los clusters sean fácilmente identificables, pero aún existe cierta variabilidad que puede resultar en clusters de tamaños similares y bien definidos.

<p align="center">  <img src="./report_gfx/NormalStd.png" width="50%"/> </p>

* **Valor de varianza alto:**<br>
Cuando los datos tienen una varianza alta, los puntos de datos se dispersan ampliamente, lo que puede dificultar la tarea del algoritmo de clustering para identificar estructuras claras. En este escenario, el algoritmo genera clusters que se ubican en áreas de baja densidad de puntos, lo que resulta en clusters que no son representativos de la verdadera estructura de los datos. Además un detalle **muy interesante** es que la alta dispersión hace que el algoritmo requiera más iteraciones para converger correctamente. Por tanto, esta falta de convergencia resulta en que aquellos puntos alejados sean clasificados parte del mismo cluster.

<p align="center">  <img src="./report_gfx/HighStd.png" width="50%"/> </p>

* **Valor de varianza muy alto:**<br>
Un valor de varianza muy alto genera una gran dispersión en los datos, lo que hace que los puntos estén extremadamente alejados unos de otros. En este caso, el algoritmo de clustering enfrenta grandes dificultades para encontrar una partición adecuada de los datos, ya que los puntos de diferentes clusters se solapan significativamente formando clusters demasiado amplios y poco definidos. Además, debido a la altísima dispersión (como anteriormente), el algoritmo necesita un número extremadamente alto de iteraciones para converger, terminando en una solución donde todos los puntos pertenecen al mismo cluster.

<p align="center">  <img src="./report_gfx/VeryHighStd.png" width="50%"/> </p>


---
# Método del codo aplicando Entropía
---

# Casos de Fallo

**1. Varianza de los datos generados**

* **Valor de varianza alto (std=5):**<br>
Cuando la varianza aumenta, los clusters dejan de ser centros compactos y comienzan a mezclarse entre sí. Esto provoca que el algoritmo ya no puede decidir con claridad a qué grupo pertenece cada punto, porque todos los centros están "compitiendo" por los mismos datos. En vez de reducirse, la entropía empieza a oscilar: a veces bajar, a veces subir. ¿Por qué? Porque añadir más clusters en este contexto no siempre aporta orden, a veces lo contrario, añade confusión dividiendo regiones que ya estaban desdibujadas. El método del codo falla aquí porque asume que más clusters siempre deben reducir la incertidumbre, pero cuando los grupos ya no existen como tales, esa lógica se rompe.

<p align="center">  <img src="./report_gfx/Elbow_5_Std.png" width="50%"/> </p>

* **Valor de varianza muy alto (std=25):**<br>
En este caso, a pesar de que con 3 clusters la entropía alcanza un mínimo aparente (bastante acertado, ya que nuestros datos fueron generados con 3 clusters). Pero al aumentar el número de clusters, la entropía se dispara de manera abrupta. Esto sucede porque, en presencia de una dispersión tan extrema, los datos pierden una estructura coherente, y el algoritmo se ve obligado a "forzar" particiones que en realidad no existen. La incorporación de clusters adicionales, con un valor de varianza tan elevado, no reduce la incertidumbre sino que la amplifica, evidenciando una debilidad del método del codo ante condiciones de dispersión extrema.

<p align="center">  <img src="./report_gfx/Elbow_25_Std.png" width="50%"/> </p>


**2. Distancia entre los centros de los clusters**

* **Centros muy cercanos entre sí:**<br> 
Cuando los centros de los clusters están demasiado próximos, el algoritmo tiene dificultades para diferenciar entre ellos. El espacio entre los centros se vuelve tan reducido que las distribuciones generadas por cada cluster se solapan casi por completo, dando la ilusión de que hay menos grupos de los que realmente existen. En consecuencia, como vemos en los gráficos obtenidos, el número óptimo de clusters que estima el método del codo comienza a fluctuar: a veces lo reduce, a veces lo aumenta según cómo se “repartan” los puntos. Esto revela un problema clave: cuando la separación entre grupos es débil, el criterio basado en entropía puede confundirse fácilmente, ya que las diferencias internas entre los clusters no son lo suficientemente marcadas como para justificar una partición clara.

<p align="center"> 
    <img src="./report_gfx/Elbow_near_centers1.png" width="30%"/>
    <img src="./report_gfx/Elbow_near_centers2.png" width="30%"/>
    <img src="./report_gfx/Elbow_near_centers3.png" width="30%"/>
</p>

---
# Ejercicio 3
---

**Datos con 3 clusters, Algoritmo con 6 clusters**: En este primer caso vemos que el algoritmo intenta identificar más clusters de los que realmente existen. Lo que podría ser útil para detectar **subgrupos** dentro de los clusters principales. En este caso, la división en 6 clusters ofrece una visión detallada que podría ser interesante para análisis más finos.

**Datos con 3 clusters, Algoritmo con 2 clusters**:  Aquí, el algoritmo subestima el número de clusters, agrupando datos dispares en conjuntos más grandes. Podemos ver que este enfoque simplifica la estructura de los datos, podríamos verlo como un **filtro**. Aunque reduce la precisión, puede ser útil yo lo utilizaría para obtener una visión general rápida de las tendencias principales.

**Datos con 5 clusters, Algoritmo con 5 clusters**: Este gráfico muestra una segmentación óptima, donde el número de clusters del algoritmo coincide con el número real de clusters en los datos. En este caso podemos comprobar el correcto funcionamiento de nuestro algoritmo.

**Datos con 5 clusters, Algoritmo con 2 clusters**: En este caso, el algoritmo reduce la complejidad al agrupar múltiples clusters en dos grandes grupos, lo que puede llevar a una pérdida de información detallada y a una interpretación menos precisa de las relaciones entre los datos. Es importante destacar la frontera visual que se genera, similar a la de un **clasificador lineal como el perceptrón**, que intenta separar los datos en el espacio de características.

**Datos con 5 clusters, Algoritmo con 10 clusters**:  Este escenario representa un caso extremo, ya que el algoritmo intenta identificar más clusters de los que existen. Sin embargo, como ya mencionamos anteriormente, puede ser útil para explorar subestructuras y **descubrir patrones ocultos** que podrían no ser evidentes a simple vista.

<p align="center">  <img src="./report_gfx/E3.png" width="70%"/> </p>


---
# Ejercicio 6
---

En este caso no logré completar el código del ejercicio. Pero realicé el reporte en base a lo que recuerdo de la clase de prácticas.

En primer lugar, la imagen corresponde a una representación visual de un tumor cerebral. El objetivo principal era procesar la imagen para posteriormente aplicar técnicas de clustering. Es decir, clasificar las zonas del cerebro en clusters, en este ejercicio se utilizaron 4 cluster, lo que también puede equivaler a 4 colores. Pero lo que nos interesa realmente es el análisis de la complejidad y los elementos visuales observables en la imagen original.

<p align="center">  
    <img src="./Tumor1.png" width="20%"/> 
    <img src="./Tumor1Segmented.png" width="20%"/> 
</p>


**1. Observaciones Visuales de la Imagen Segmentada:**<br>
La imagen presenta un contraste moderado, especialmente en las zonas más centrales donde se distinguen diversas estructuras cerebrales como el núcleo, la corteza y el posible tejido tumoral. Si bien la segmentación ha podido separar visualmente varias regiones, existen errores notables en áreas donde los bordes entre tejidos son poco definidos. En general, la segmentación se ve comprometida en zonas de transición tonal suave.

Uno de los aspectos relevantes al observar la imagen segmentada es que la región correspondiente al tumor cerebral, ubicada a la izquierda del centro de la imagen, ha sido agrupada dentro de un **clúster específico** de tonalidad más clara. Este clúster resalta el área sospechosa de forma bastante efectiva.

Sin embargo, se puede observar que otras zonas, especialmente el pellejo o borde externo del cerebro, también fueron clasificadas dentro del mismo clúster que el tumor. Esto ocurre porque estas regiones comparten características visuales similares, además hay que tener en cuenta el algoritmo no tiene un conocimiento anatómico que le permita discernir de forma tan precisa y también ha sido limitado en el uso de **4 clusters**, lo que implica menor precisión.

También podemos observar zonas negras dentro del cerebro, lo que indica regiones que fueron agrupadas con el cluster del exterior, seguramente debido a su baja intensidad o falta de información visual. Además, dentro del propio tumor, se distingue una zona gris interna, lo que evidencia que el tumor no es completamente uniforme. Esta variación tonal sugiere diferencias internas en la densidad o estructura del tejido, algo que podría tener importancia para un médico debido a la composición del tumor.


**2. Elementos Perturbadores en la Imagen**<br>
La imagen contiene además una marca de agua ubicada en la parte superior derecha, así como una línea fina que cubre el borde derecho de la imagen que corresponde probablemente a un márgen del corte realizado. Estas zonas ajenas al tejido cerebral pueden ser confundidas por el algoritmo como clústeres relevantes, y por tanto, para otros algoritmos podría ser una fuente muy relevante en cuánto a clasificación de imágenes si tenemos en cuenta que todas las imágenes con tumores tienen la misma marca de agua. Por tanto es recomendable eliminar previamente esta marca de agua para procesamientos más fiables.

**3. Consideraciones Finales**
La segmentación realizada sobre la imagen del tumor cerebral logra captar varias estructuras relevantes, incluyendo la masa tumoral. No obstante, debido a la complejidad tonal y la ausencia de información anatómica contextual en el proceso, es comprensible que otras regiones como el borde del cerebro hayan sido agrupadas en el mismo clúster. Esto pone de relieve la necesidad, de incorporar modelos supervisado, y de la eliminación previa de marcas para excluir regiones y patrones no deseados.

### <center>Santiago Álvarez Geanta</center>
#### <center>Universidad de Alicante</center>
<p align="center"><img src=".ipynb_gfx/UA.png" width="165" height="95"></p>